<a href="https://colab.research.google.com/github/fininsight/text-mining-tutorial/blob/master/1_%EB%8B%A8%EC%96%B4%EC%9D%98_%ED%91%9C%ED%98%84_Word_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 단어의 표현 (Word Representation)

## 1 원-핫 인코딩 (One-Hot Encoding)

### "원숭이, 바나나, 사과" 로 원-핫 인코딩을 한다면

In [0]:
# 인코딩 대상 단어들을 담은 리스트
word_ls = ['원숭이','바나나','사과']

In [0]:
from collections import defaultdict
import numpy as np 

def one_hot_encode(word_ls):
    # 고유 단어와 인덱스를 매칭시켜주는 사전 생성
    word2idx_dic = defaultdict(lambda:len(word2idx_dic)) 
    
    # {단어 : 인덱스} 사전 구축
    for word in word_ls:
        word2idx_dic[word]
    
    n_unique_words = len(word2idx_dic) # 고유한 단어의 갯수
    one_hot_vectors = np.zeros((len(word_ls), n_unique_words)) # 원핫-벡터를 만들기 위해 비어있는 벡터 생성
    
    for i,word in enumerate(word_ls):
        index = word2idx_dic[word] # 해당 단어의 고유 인덱스
        one_hot_vectors[i, index] += 1 # 해당 단어의 고유 인덱스에만 1을 더해줌
        
    return one_hot_vectors

In [0]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

### "코끼리"라는 단어가 추가된다면?

In [0]:
word_ls = ['원숭이','바나나','사과','코끼리']

In [0]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## 2 밀집 벡터 (Dense Vector)

In [0]:
import numpy as np

In [0]:
word_embedding_dic = {
    '사과' : [1.0, 0.5],
    '바나나' : [0.9, 1.2],
    '원숭이' : [0.5, 1.5]
}

### 2.1 코사인 유사도(Cosine Similarity) 계산

In [0]:
def cal_cosine_similarity(x, y):
    # x와 y, 두 벡터의 코사인 유사도를 계산하는 함수
    nominator = np.dot(x, y)    # 분자
    denominator = np.linalg.norm(x)*np.linalg.norm(y)  # 분모
    return nominator/denominator

In [0]:
# 사과와 바나나의 코사인 유사도
cal_cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['바나나'])

0.8944271909999159

In [0]:

# 사과와 원숭이의 코사인 유사도
cal_cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['원숭이'])

0.7071067811865475

In [0]:
# 바나나와 원숭이의 코사인 유사도
cal_cosine_similarity(word_embedding_dic['바나나'], word_embedding_dic['원숭이'])

0.9486832980505138

## 3 단어 임베딩 (Word Embedding)

### 3.1 구글드라이브 내 파일 읽어오기

In [0]:
# 구글 드라이브에서 csv 파일을 읽어오기 위해 gauth 인증을 합니다.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
url_it ='https://drive.google.com/open?id=1Yj9sJ-E_ZiOmEcHpsHI7-iIvs08HEVNN'
id_it = url_it.split('=')[1]
print(id_it)

url_culture ='https://drive.google.com/open?id=1eG0Tdh3b27Su5Zt45LXdBS3sHe2kDb7x'
id_culture = url_culture.split('=')[1]
print(id_culture)

1Yj9sJ-E_ZiOmEcHpsHI7-iIvs08HEVNN
1eG0Tdh3b27Su5Zt45LXdBS3sHe2kDb7x


In [0]:
%ls

adc.json  실리콘벨리기사_전처리.csv  sample_data/


In [0]:
# 크롤링해 온 국민청원 데이터를 판다스를 통해 읽어온다.
downloaded = drive.CreateFile({'id':id_it}) 
downloaded.GetContentFile('2_word_embedding_it.csv')  

# 크롤링해 온 국민청원 데이터를 판다스를 통해 읽어온다.
downloaded = drive.CreateFile({'id':id_culture}) 
downloaded.GetContentFile('2_word_embedding_culture.csv')  

In [0]:
%ls

'2_word embedding_culture.csv'  '2_word embedding_it.csv'   adc.json
 2_word_embedding_culture.csv    2_word_embedding_it.csv    sample_data/


In [0]:
import pandas as pd
it_df = pd.read_csv('2_word_embedding_it.csv').dropna()
culture_df = pd.read_csv('2_word_embedding_culture.csv').dropna()

ParserError: ignored

### 3.2 Word2Vec

In [0]:
it_token_ls = list(it_df.loc[:,'1'])
culture_token_ls = list(culture_df.loc[:,'1'])
total_token_ls = it_token_ls + culture_token_ls

In [0]:
total_token_ls[0]

In [0]:
total_token_ls = [tokens.split(',') for tokens in total_token_ls]

In [0]:
total_token_ls[0][:10]

In [0]:

from gensim.models import Word2Vec

In [0]:

word2vec = Word2Vec(
    sentences = total_token_ls,  # 학습시킬 문장
    size = 30,                  # 임베딩된 단어 벡터의 차원 크기
    alpha = 0.025,               # 학습률(Learning rate)
    min_count=2,                 # 1번 미만 등장한 단어는 제외
    window = 8,                  # 문맥의 크기 (window_size)
    sample = 0.001,              # sub-sampling
    sg = 1,                      # 0: CBOW, 1: Skip-gram
    iter = 10,                    # 전체 문장 반복학습 횟수(epoch)
    )

In [0]:
word2vec.wv.__getitem__('문화')

In [0]:
word2vec.wv.most_similar('인공')

In [0]:
word2vec.wv.most_similar('공연')

In [0]:
culture_df

### 3.3 차원축소 활용 시각화